<a href="https://www.kaggle.com/code/istyfatinfaiaz/sentence-prediction-roberta?scriptVersionId=111103715" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentence-prediction-dataset/FAQs_test.csv
/kaggle/input/sentence-prediction-dataset/FAQs.csv


In [2]:
faq   = pd.read_csv("../input/sentence-prediction-dataset/FAQs.csv")
faq_t = pd.read_csv("../input/sentence-prediction-dataset/FAQs_test.csv") 
faq_ques = faq["Question"].values.tolist()
faq_ans  = faq["Answer"].values.tolist()
faq_test = faq_t["Question"].values.tolist()

In [3]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9e7e6a33b837ae73b3a2b779d32800705f4be93dc54e2fb6a98c5ad3a1f85472
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Finding Answer through Text Similarity
We'll use Cosine Similarity to check sentence similarity to answer question.

In [4]:
from sentence_transformers import SentenceTransformer
model1 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
faq_ques_embed = model1.encode(faq_ques)
faq_ans_embed  = model1.encode(faq_ans)
faq_t_embed    = model1.encode(faq_test)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(faq_t_embed, faq_ans_embed)

similarity_matrix

array([[0.31960434, 0.37441796, 0.45771325, 0.2538718 , 0.23105325,
        0.2817713 , 0.23049985, 0.17161308, 0.28659773, 0.11337139],
       [0.53099245, 0.40804985, 0.1855129 , 0.6576111 , 0.48623058,
        0.43744034, 0.23440342, 0.2821349 , 0.35925895, 0.4243167 ],
       [0.1350576 , 0.25017792, 0.25782213, 0.38549954, 0.43349272,
        0.5667815 , 0.18126853, 0.11547635, 0.11563492, 0.17477426],
       [0.52820015, 0.45062608, 0.24976528, 0.74789774, 0.32070357,
        0.28186238, 0.2145313 , 0.32090193, 0.3580909 , 0.48909622],
       [0.24043944, 0.31182605, 0.30395457, 0.29077703, 0.16944996,
        0.13972111, 0.5887659 , 0.27596778, 0.24217609, 0.3425349 ]],
      dtype=float32)

In [7]:
index = similarity_matrix.argmax(axis=1)
index

array([2, 3, 5, 3, 6])

In [8]:
ans_list = []
for i in index:
    ans_list.append(faq_ans[i])

ans_list = pd.DataFrame(ans_list, columns=["FAQs_Answer"])
ans_list

,FAQs_Answer
0,"He died 18 April 1955 in Princeton, New Jersey..."
1,His father was Hermann Einstein and his mother...
2,He was married to Mileva Marić between 1903 an...
3,His father was Hermann Einstein and his mother...
4,He received his main education at the followin...


In [9]:
ques_test = pd.DataFrame({"FAQs_test":faq_test})
ques_test

,FAQs_test
0,What is the date of his death?
1,Did Einstein have siblings?
2,Who was his wife?
3,What was Einstein's father's name?
4,At what institutions did he study?


**Here we see whether the model can predict the answers**

In [10]:
que_ans = pd.concat([ques_test, ans_list], axis=1, join='inner')
que_ans

,FAQs_test,FAQs_Answer
0,What is the date of his death?,"He died 18 April 1955 in Princeton, New Jersey..."
1,Did Einstein have siblings?,His father was Hermann Einstein and his mother...
2,Who was his wife?,He was married to Mileva Marić between 1903 an...
3,What was Einstein's father's name?,His father was Hermann Einstein and his mother...
4,At what institutions did he study?,He received his main education at the followin...


# roBERTa Similarity Testing
We check whether the question similarity is seen here

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

In [12]:
model = SentenceTransformer('xlm-roberta-base')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
embedding1 = model.encode(faq_ques, convert_to_tensor=True)
embedding2 = model.encode(faq_test, convert_to_tensor=True)

cosine_scores = util.pytorch_cos_sim(embedding2, embedding1)
max_sim     = -1.0
max_sim_mat = []
for i in range(len(faq_ques)):
    if (max_sim != -1.0):
        max_sim_mat.append(max_sim)
    max_sim = -1.0
    for j in range(len(faq_test)):
        print("Question "+str(i+1)+":", faq_ques[i])
        print("Test Question "+str(j+1)+":", faq_test[j])
        if (cosine_scores[j][i] > max_sim):
            max_sim     = cosine_scores[j][i]
        print("Similarity Score:", cosine_scores[j][i].item())
        print()
print(max_sim_mat)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Question 1: When was Albert Einstein born?
Test Question 1: What is the date of his death?
Similarity Score: 0.9961808323860168

Question 1: When was Albert Einstein born?
Test Question 2: Did Einstein have siblings?
Similarity Score: 0.9967830181121826

Question 1: When was Albert Einstein born?
Test Question 3: Who was his wife?
Similarity Score: 0.9971728324890137

Question 1: When was Albert Einstein born?
Test Question 4: What was Einstein's father's name?
Similarity Score: 0.9951794147491455

Question 1: When was Albert Einstein born?
Test Question 5: At what institutions did he study?
Similarity Score: 0.9964344501495361

Question 2: Where was he born?
Test Question 1: What is the date of his death?
Similarity Score: 0.9904046654701233

Question 2: Where was he born?
Test Question 2: Did Einstein have siblings?
Similarity Score: 0.9907991886138916

Question 2: Where was he born?
Test Question 3: Who was his wife?
Similarity Score: 0.9940136075019836

Question 2: Where was he bor

# Predicting Test Question Answer using the Answers
In this section, we give the model a context text and based on that it can predict the answer to those questions.
Note: The context is basically the answers from FAQs List.

In [14]:
string = " ".join(faq_ans)
string

'Albert Einstein was born on 14 March 1879. He was born in Ulm, Germany.  He died 18 April 1955 in Princeton, New Jersey, USA. His father was Hermann Einstein and his mother was Pauline Einstein (born Koch). He had one sister named Maja. He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936. He received his main education at the following schools: Catholic elementary school in Munich, Germany (1885-1888)Luitpold Gymnasium in Munich, Germany (1888-1894) Cantonal school in Aarau, Switzerland (1895-1896) Swiss Federal Institute of Technology in Zurich, Switzerland (1896-1900) Ph.D. from Zurich University, Switzerland (1905) The Nobel Prize Awarding Institution, the Royal Swedish Academy of Sciences, decided to reserve the Nobel Prize in Physics in 1921, and therefore no Physics Prize was awarded that year. According

In [15]:
context = string

In [16]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [17]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [18]:
def question_answer(question, text):
    
    #tokenize question and text as a pair
    input_ids = tokenizer.encode(question, text)
    
    #string version of tokenized ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    #segment IDs
    #first occurence of [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    #number of tokens in segment A (question)
    num_seg_a = sep_idx+1
    #number of tokens in segment B (text)
    num_seg_b = len(input_ids) - num_seg_a
    
    #list of 0s and 1s for segment embeddings
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    
    #model output using input_ids and segment_ids
    output = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]))
    
    #reconstructing the answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end >= answer_start:
        answer = tokens[answer_start]
        for i in range(answer_start+1, answer_end+1):
            if tokens[i][0:2] == "##":
                answer += tokens[i][2:]
            else:
                answer += " " + tokens[i]
                
    if answer.startswith("[CLS]"):
        answer = "Unable to find the answer to your question."
    
    print("\nPredicted answer:\n{}".format(answer.capitalize()))

In [19]:
for each in faq_test:
    print("Question: ", each)
    print(question_answer(each, context))
    print("\n")

Question:  What is the date of his death?

Predicted answer:
18 april 1955
None


Question:  Did Einstein have siblings?

Predicted answer:
Maja
None


Question:  Who was his wife?

Predicted answer:
Elsa lowenthal
None


Question:  What was Einstein's father's name?

Predicted answer:
Hermann einstein
None


Question:  At what institutions did he study?

Predicted answer:
Swiss federal institute of technology in zurich , switzerland
None




In [20]:
for each in faq_ques:
    print("Question: ", each)
    print(question_answer(each, context))
    print("\n")

Question:  When was Albert Einstein born?

Predicted answer:
14 march 1879
None


Question:  Where was he born?

Predicted answer:
Ulm , germany
None


Question:  When did he die?

Predicted answer:
18 april 1955
None


Question:  Who were his parents?

Predicted answer:
Hermann einstein and his mother was pauline einstein ( born koch )
None


Question:  Did he have any sisters and brothers?

Predicted answer:
He had one sister named maja
None


Question:  Did he marry and have children?

Predicted answer:
He was married to mileva maric between 1903 and 1919 . they had three children , lieserl ( born 1902 ) , hans albert ( born 1904 ) and eduard ( born 1910 )
None


Question:  Where did he receive his education?

Predicted answer:
Catholic elementary school in munich , germany
None


Question:  When was Albert Einstein awarded the Nobel Prize in Physics?

Predicted answer:
1922
None


Question:  Did Albert Einstein attend the Nobel Prize Award Ceremony?

Predicted answer:
Could not
Non